In [1]:
from datasets import load_dataset
dataset = load_dataset("multi_news")
ex = dataset["validation"][0]
docs = ex["document"].split('|||||')
print(len(docs))
for d in docs:
    print("DOC: " ,d)

/afs/crc.nd.edu/user/m/myu2/anaconda2/envs/bert/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using custom data configuration default
Reusing dataset multi_news (/afs/crc.nd.edu/user/m/myu2/.cache/huggingface/datasets/multi_news/default/1.0.0/2f1f69a2bedc8ad1c5d8ae5148e4755ee7095f465c1c01ae8f85454342065a72)
100%|██████████| 3/3 [00:00<00:00, 485.73it/s]

4
DOC:  Whether a sign of a good read; or a comment on the 'pulp' nature of some genres of fiction, the Oxfam second-hand book charts have remained in The Da Vinci Code author's favour for the past four years. 
 
 Dan Brown has topped Oxfam's 'most donated' list again, his fourth consecutive year. Having sold more than 80 million copies of The Da Vinci Code and had all four of his novels on the New York Times bestseller list in the same week, it's hardly surprising that Brown's hefty tomes are being donated to charity by readers keen to make some room on their shelves. 
 
 Another cult crime writer responsible to heavy-weight hardbacks, Stieg Larsson, is Oxfam's 'most sold' author for the second time in a row. Both the 'most donated' and 'most sold' lists are dominated by crime fiction, trilogies and fantasy, with JK Rowling the only female author listed in either of the Top Fives. 
 
 Click here or on "View Gallery" to see both charts in pictures 
DOC:   A woman reads a copy of the ne

In [2]:
ex["summary"]

'– The Da Vinci Code has sold so many copies—that would be at least 80 million—that it\'s bound to turn up in book donation piles. But at one charity shop in the UK, it\'s been donated so heavily that the shop has posted a sign propped up on a tower of Da Vinci Code copies that reads: "You could give us another Da Vinci Code... but we would rather have your vinyl!" The manager of the Oxfam shop in Swansea tells the Telegraph that people are laughing and taking pictures of the sizable display: "I would say that we get one copy of the book every day." He says people buy them "occasionally," but with vinyl sales up 25% in the past year, they\'d rather take records. Dan Brown\'s book isn\'t the only one that shops like Oxfam struggle to re-sell. Last year, Oxfam was hit with a large and steady supply of Fifty Shades of Grey, and it similarly begged donors: "Please—no more." But Brown has a particular kind of staying power. The Da Vinci Code was published in 2003, and within six years Brown

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from itertools import product, combinations

from nltk import sent_tokenize

all_sentences = []
id2sent = {}
sent2id = {}
for i,d in enumerate(docs):
    sentences = sent_tokenize(d)
    for j,sent in enumerate(sentences):
        id2sent[f"{i}_{j}"] = sent
    all_sentences.append([x.strip() for x in sentences])
    

model_dir =  "/afs/crc.nd.edu/group/dmsquare/vol2/myu2/ComparisonSentences/experiments/pairwise_cls/cross_enc_combined_cross0_1120/checkpoint-5802"
model = AutoModelForSequenceClassification.from_pretrained(
    model_dir
)
tokenizer = AutoTokenizer.from_pretrained(model_dir)

results = []
for docid_1, docid_2 in combinations(range(len(docs)), 2):
    for sentid_1, sentid_2 in product(range(len(all_sentences[docid_1])), range(len(all_sentences[docid_2]))):
        t1 = id2sent[f"{docid_1}_{sentid_1}"]
        t2 = id2sent[f"{docid_2}_{sentid_2}"]
        inputs = tokenizer(
            t1, t2,
            return_tensors="pt"
        )
        # print(inputs)
        with torch.no_grad():
            outputs = model(**inputs)

            label = int(torch.argmax(outputs.logits))
            # print(label)
            obj = {
                "t1_id": f"{docid_1}_{sentid_1}",
                "t2_id": f"{docid_2}_{sentid_2}",
                "t1": t1,
                "t2": t2,
                "prediction": label
            }
            results.append(obj)

In [4]:
len(results)

1437

In [7]:
import json
json.dump(results, open("multi-news_dev_0_results.json", 'w'))

In [8]:
from collections import Counter
cnt = Counter()
for r in results:
    if r["prediction"] == 1:
        cnt[r["t1_id"]] += 1
        cnt[r["t2_id"]] += 1
for k,v in cnt.most_common(5):
    print(k,v, id2sent[k])

1_10 35 It follows the adventures of Harvard professor Robert Langdon and is set in the secret world of Freemasons in Washington D.C. 
 
 "The Da Vinci Code" sold 80 million copies worldwide.
1_1 34 REUTERS/Tim Wimborne 
 
 SAN FRANCISCO The latest novel from "Da Vinci Code" author Dan Brown, "The Lost Symbol," broke one-day sales records, its publisher and booksellers said.
0_0 28 Whether a sign of a good read; or a comment on the 'pulp' nature of some genres of fiction, the Oxfam second-hand book charts have remained in The Da Vinci Code author's favour for the past four years.
3_1 26 The Oxfam shop in Swansea has been receiving an average of one copy of the Dan Brown novel a week for months, leaving them with little room for any other books.
0_3 21 Another cult crime writer responsible to heavy-weight hardbacks, Stieg Larsson, is Oxfam's 'most sold' author for the second time in a row.


In [11]:
from rouge_score import rouge_scorer
pred = ' '.join([id2sent[k] for k,v in cnt.most_common(10)])
ref = ex["summary"].lower()
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores = scorer.score(pred,
                      ref)
print(scores)

{'rouge1': Score(precision=0.5019762845849802, recall=0.4471830985915493, fmeasure=0.47299813780260713), 'rouge2': Score(precision=0.14682539682539683, recall=0.13074204946996468, fmeasure=0.13831775700934582), 'rougeL': Score(precision=0.23715415019762845, recall=0.2112676056338028, fmeasure=0.22346368715083795)}


# Construct dataset for pairwise comparison

In [1]:
from datasets import load_dataset
from itertools import product, combinations
from nltk import sent_tokenize
import json
dataset = load_dataset("multi_news")

print("data size - train/dev/test: ", len(dataset["train"]), len(dataset["validation"]), len(dataset["test"]))


split = "test"
fw = open(f"multi-news_pairwise/data_{split}.json", 'w')
cnt_sentence = 0
for exp_id, ex in enumerate(dataset[split]):
    doc_sentences = [] # list of list of sentences
    id2sent = {}
    docs = ex["document"].split('|||||')
    for i,d in enumerate(docs):
        sentences = sent_tokenize(d)
        for j,sent in enumerate(sentences):
            id2sent[f"{i}_{j}"] = sent
        doc_sentences.append([x.strip() for x in sentences])

    
    for docid_1, docid_2 in combinations(range(len(docs)), 2):
        for sentid_1, sentid_2 in product(range(len(doc_sentences[docid_1])), range(len(doc_sentences[docid_2]))):
            t1 = id2sent[f"{docid_1}_{sentid_1}"]
            t2 = id2sent[f"{docid_2}_{sentid_2}"]
            if t1.count(' ') < 2 or t2.count(' ') < 2:
                continue # don't need "too short sentences
            obj = {
                "id": f"{exp_id}|{docid_1}_{sentid_1}|{docid_2}_{sentid_2}",
                "t1": t1,
                "t2": t2,
                "label": 0 # this is a fake label
            }
            # data.append(obj)
            fw.write(json.dumps(obj) + '\n')
            cnt_sentence += 1
    
fw.close()
print("cnt sentences:", cnt_sentence)



/afs/crc.nd.edu/user/m/myu2/anaconda2/envs/bert/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using custom data configuration default
Reusing dataset multi_news (/afs/crc.nd.edu/user/m/myu2/.cache/huggingface/datasets/multi_news/default/1.0.0/2f1f69a2bedc8ad1c5d8ae5148e4755ee7095f465c1c01ae8f85454342065a72)
100%|██████████| 3/3 [00:00<00:00,  7.21it/s]


data size - train/dev/test:  44972 5622 5622
cnt sentences: 17396130
